In [23]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import json
from google.cloud import storage
from google.cloud import aiplatform as vertex_ai

import os

input_bucket_path = "gs://berkabank/production/data/"

In [24]:
import joblib

bucket_name = "berkabank"
source_blob_name = "production/artifacts/model/berkamodel.joblib"
destination_file_name = "./berkamodel.joblib"
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(source_blob_name)
blob.download_to_filename(destination_file_name)



In [25]:
joblib.load(destination_file_name).predict_proba([[3]]) 

/Users/davidelupis/Desktop/Testing/berkabank/.venv/lib/python3.9/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([[0.64975432, 0.35024568]])

In [26]:
# Authenticate with Google Cloud SDK for Vertex AI
PROJECT_ID = os.environ.get("PROJECT_ID")
REGION = os.environ.get("REGION")
REPOSITORY = os.environ.get("BUCKET_NAME")  # Match the Bucket name on Artifact Registry
# Define the pipeline name
PIPELINE_NAME = "production"

# Defautl arguments for the pipeline
default_params = {
    "project_id": os.environ.get("PROJECT_ID"),
    "region": os.environ.get("REGION"),
    "bucket_name": os.environ.get("BUCKET_NAME"),
    "pipeline_name": PIPELINE_NAME,
}

# ------------- Define the details component -------------
BASE_IMAGE = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{PIPELINE_NAME}:latest"

# Init VertexAI
vertex_ai.init(project=PROJECT_ID, location=REGION)

In [27]:
joblib.load(destination_file_name).feature_names_in_

array(['n_transactions'], dtype=object)

In [28]:
from google.cloud import aiplatform as vertex_ai

vertex_ai.init(project=PROJECT_ID, location=REGION)
VERTEX_AI_MODEL_ID = "berkamodel"

upload_config = vertex_ai.Model.upload(
    display_name=VERTEX_AI_MODEL_ID,
    is_default_version=False,
    version_aliases=["experimental", "challenger", "custom-training", "random-forest"],
    version_description="A classifier",
    serving_container_image_uri=BASE_IMAGE,
    serving_container_health_route="/health",
    serving_container_predict_route="/predict",
    serving_container_ports=[8080],
    labels={"created_by": "davide", "team": "badass"},
)

Creating Model
Create Model backing LRO: projects/1036389498447/locations/europe-west6/models/783890217953132544/operations/3688180913490886656
Model created. Resource name: projects/1036389498447/locations/europe-west6/models/783890217953132544@1
To use this Model in another session:
model = aiplatform.Model('projects/1036389498447/locations/europe-west6/models/783890217953132544@1')


In [29]:
# Create Endpoint
model_name = "berkamodel"
model_endpoint = f"{model_name}_endpoint"
# Fetch existing endpoint if exist
endpoints = vertex_ai.Endpoint.list(
    filter='display_name="{}"'.format(model_endpoint),
    order_by="create_time desc",
    project=PROJECT_ID,
    location=REGION,
)
# If endpoint exists take most recent otherwise create endpoint
# If endpoint exists take most recent otherwise create endpoint
print(endpoints)
if len(endpoints) > 0:
    endpoint = endpoints[0]  # most recently created
else:
    # Create Endpoint
    endpoint = vertex_ai.Endpoint.create(
        display_name=model_endpoint, project=PROJECT_ID, location=REGION
    )

resource name: projects/1036389498447/locations/europe-west6/endpoints/1986360114554077184]


In [30]:
# Make Champion Model and Deploy to Endpoint
deployed_model_display_name = f"{model_name}_champion"
endpoint_config = upload_config.deploy(
    deployed_model_display_name=deployed_model_display_name,
    endpoint=endpoint,
    min_replica_count=1,
    max_replica_count=1,
)

Deploying model to Endpoint : projects/1036389498447/locations/europe-west6/endpoints/1986360114554077184
Using default machine_type: n1-standard-2
Deploy Endpoint model backing LRO: projects/1036389498447/locations/europe-west6/endpoints/1986360114554077184/operations/8203039539929808896
Endpoint model deployed. Resource name: projects/1036389498447/locations/europe-west6/endpoints/1986360114554077184
